In [146]:
import os
import argparse
import numpy as np
from time import time
from collections import defaultdict

BASE_LOC = r'/home/or/dev/Intro_to_NLP/task2'
POS_DATA_LOC = os.path.join(BASE_LOC, r'data/pos')
POS_TRAIN_FILE = os.path.join(POS_DATA_LOC, 'ass1-tagger-train')
POS_DEV_FILE = os.path.join(POS_DATA_LOC, 'ass1-tagger-dev')
#POS_TEST_FILE = os.path.join(POS_DATA_LOC, 'ass1-tagger-test-input')

NONE_EXIST = -1
NONE_POS_IND = 0
LEFT_POS_IND = 1
RIGHT_POS_IND = 2
BOTH_POS_IND = 3


In [47]:
def add_wp_to_dict(the_dict, word, pos):
    try:
        if word in the_dict:
            if pos in the_dict[word]:
                the_dict[word][pos]['wp_count'] += 1
            else:
                the_dict[word][pos] = {'wp_count':1, 'right_pos':{}, 'left_pos':{}}

        else:
            the_dict[word] = {pos: {'wp_count':1, 'right_pos':{}, 'left_pos':{}}}
    except:
        import pdb; pdb.set_trace();

def add_right_pos_to_dict(the_dict, word, pos, right_pos):
    try:
        #import pdb; pdb.set_trace();
        if right_pos in the_dict[word][pos]['right_pos']:
            the_dict[word][pos]['right_pos'][right_pos]['right_count'] += 1
        else:
            the_dict[word][pos]['right_pos'][right_pos] = {'right_count':1, 'left_pos':{}}
    except:
        import pdb; pdb.set_trace();

def add_left_pos_to_dict(the_dict, word, pos, left_pos):
    if left_pos in the_dict[word][pos]['left_pos']:
        the_dict[word][pos]['left_pos'][left_pos]['left_count'] += 1
    else:
        the_dict[word][pos]['left_pos'][left_pos] = {'left_count':1}
        
def add_wp_right_pos_left_pos_to_dict(the_dict, word, pos, right_pos, left_pos):
    try:    
        if left_pos in the_dict[word][pos]['right_pos'][right_pos]['left_pos']:
            the_dict[word][pos]['right_pos'][right_pos]['left_pos'][left_pos]['left_count'] += 1
        else:
            the_dict[word][pos]['right_pos'][right_pos]['left_pos'][left_pos] = {'left_count':1} 
    except:
        import pdb; pdb.set_trace();        
def split_token_pos(token_and_pos, loc):
    try:    
        token, pos = token_and_pos[loc].rsplit('/',1)
    except:
        import pdb; pdb.set_trace()
    return token, pos



def read_file(loc):
    file = open(POS_TRAIN_FILE, "r")
    file_lines = file.readlines()
    file.close()
    return file_lines

def create_dict():
    file_lines = read_file(POS_TRAIN_FILE)
    words_pos_dict = defaultdict()
    for file_line in file_lines:
        splitted_line = file_line.split()
        len_line = len(splitted_line)
        for ii, _ in enumerate(splitted_line):
            token, pos = split_token_pos(splitted_line, ii)
            add_wp_to_dict(words_pos_dict, token, pos)
            if ii  + 1 < len_line:
                right_token, right_pos = split_token_pos(splitted_line, ii + 1)
                add_right_pos_to_dict(words_pos_dict, token, pos, right_pos)
            if ii - 1 <= 0:
                left_token, left_pos = split_token_pos(splitted_line, ii - 1)
                add_left_pos_to_dict(words_pos_dict, token, pos, left_pos)
            if (ii - 1 <= 0) and (ii  + 1 < len_line):
                add_wp_right_pos_left_pos_to_dict(words_pos_dict, token, pos, right_pos, left_pos)
    return words_pos_dict

    file_lines = read_file(POS_TRAIN_FILE)

words_pos_dict = create_dict()
read_file(loc)
#POS_DEV_FILE

In [147]:
file_lines = read_file(POS_DEV_FILE)

In [163]:
import json

with open(os.path.join(BASE_LOC, r'dict_temp.json'), 'w') as fp:
    json.dump(words_pos_dict, fp)

In [148]:
def possible_location(annotated_location):
    len_line = len(annotated_location)
    dist_tokens = np.ones(len_line) * -1

    for loc in range(len_line):
        if annotated_location[loc] == 0:
            loc_value = 0
            if loc + 1 < len_line:
                if annotated_location[loc + 1] == 1:
                    loc_value += 2

            if loc - 1 >= 0:
                if annotated_location[loc -1] == 1:
                    loc_value += 1

            dist_tokens[loc] = loc_value
    print(dist_tokens)
    return dist_tokens

def split_to_lists(line):
    words_list =[]
    pos_list = []
    
    for token_and_pos in line.split():
        token, pos = token_and_pos.rsplit('/',1)
        words_list.append(token)
        pos_list.append(pos)
        
    return words_list, pos_list

In [150]:
def match_value(the_dict, indication, right_pos, left_pos):
    gotten_pos_val = NONE_EXIST
    if indication == BOTH_POS_IND:
        if right_pos in the_dict['right_pos'] and \
        left_pos in the_dict['right_pos'][right_pos]['left_pos']:
            gotten_pos_val = the_dict['right_pos'][right_pos]['left_pos'][left_pos]['left_count']
        
    if indication == RIGHT_POS_IND:
        if right_pos in the_dict['right_pos']:
            gotten_pos_val = the_dict['right_pos'][right_pos]['right_count']
        
    if indication == LEFT_POS_IND:
        if left_pos in the_dict['left_pos']:
            gotten_pos_val = the_dict['left_pos'][left_pos]

    if indication == NONE_POS_IND:
        gotten_pos_val = the_dict['wp_count']
        
    return gotten_pos_val

def best_pos_in_loc(loc, the_dict, words_list, pos_list, indicaton):
    best_pos = ''
    best_count = NONE_EXIST
    
    if words_list[loc] in the_dict.keys():
        right_pos = pos_list[loc + 1]
        left_pos = pos_list[loc - 1]
        
        for pos, pos_values in the_dict[words_list[loc]].items():
            val = match_value(the_dict, indication, right_pos,left_pos)
            if best_count < val:
                best_count = val
                best_pos = pos

    return best_count, best_pos

In [161]:
def get_best_pos(file_lines, annotated_location, words_list, pos_list, the_dict):
    possible_locs = possible_location(annotated_location)


    best_count, best_pos, loc = get_pos_for_indication(possible_locs, BOTH_POS_IND, the_dict, words_list, pos_list)
    if best_count != NONE_EXIST:
        return best_pos, loc
    best_count, best_pos, loc = get_pos_for_indication(possible_locs, RIGHT_POS_IND, the_dict, words_list, pos_list)
    if best_count != NONE_EXIST:
        return best_pos, loc
    best_count, best_pos, loc = get_pos_for_indication(possible_locs, LEFT_POS_IND, the_dict, words_list, pos_list)
    if best_count != NONE_EXIST:
        return best_pos, loc
    best_count, best_pos, loc = get_pos_for_indication(possible_locs, NONE_POS_IND, the_dict, words_list, pos_list)
    return best_pos, loc

def get_pos_for_indication(all_possible_locs, indication, the_dict, words_list, pos_list):
    specific_possible_locs = np.where(all_possible_locs == indication)

    best_pos = ''
    best_count = NONE_EXIST

    for loc in specific_possible_locs:
        tested_pos, tested_val = best_pos_in_loc(loc, the_dict, words_list, pos_list, indication)
        if tested_pos != NONE_EXIST:
            if  best_count < tested_val:
                best_count = tested_val
                best_pos = tested_pos

    return best_count, best_pos, loc

In [162]:
splitted_line = file_lines[0].split()
len_line = len(splitted_line)
annotated_location = np.zeros(len_line)
annotated_token = [''] * len_line
words_list = []
pos_lost = []

words_list, label_pos = split_to_lists(file_lines[0])
while not all(annotated_location):
    best_pos, loc = get_best_pos(file_lines, annotated_location, words_list, annotated_token, words_pos_dict)
    annotated_location[loc] = 1
    annotated_token[loc] = best_pos

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.]


TypeError: only integer scalar arrays can be converted to a scalar index

In [113]:
# def location_to_anottate(annotated_location):
#     len_line = len(annotated_location)
#     dist_tokens = np.ones(len_line) * -1

#     for loc in range(len_line):
#         if annotated_location[loc] == 0:
#             loc_value = 0
#             if loc + 1 < len_line:
#                 if annotated_location[loc + 1] == 1:
#                     loc_value += 2

#             if loc - 1 >= 0:
#                 if annotated_location[loc -1] == 1:
#                     loc_value += 1

#             if loc_value == 3:
#                 return loc, loc_value

#             dist_tokens[loc] = loc_value

#     index = np.argmax(dist_tokens)
#     val_max = dist_tokens[index]
#     return index, val_max

In [ ]:
# if kind == NONE_POS_IND:
#     if token in words_pos_dict.keys():
#         max_count = -np.inf
#         selected_pos = ''
#         for tested_pos, pos_values in words_pos_dict[token].items():
#             wp_count = pos_values['wp_count']
#             if wp_count > max_count:
#                 max_count = wp_count 
#                 selected_pos = tested_pos
#     else:
#         raise token
        
# if kind == LEFT_POS_IND:
#     if token in words_pos_dict.keys():
#     max_count = -np.inf
#     selected_pos = ''
#     for tested_pos, pos_values in words_pos_dict[token].items():
#         for
#         wp_count = pos_values['wp_count']
#         if wp_count > max_count:
#             max_count = wp_count 
#             selected_pos = tested_pos
#     else:
#         raise token